## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
model_data.dtypes

Angaus          int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method         object
LocSed        float64
dtype: object

In [3]:
le = LabelEncoder()
model_data['Method'] = le.fit_transform(model_data['Method'].astype(str))

X = model_data.drop("Angaus", axis=1)
y = model_data["Angaus"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=808)

model_data.dtypes

Angaus          int64
SegSumT       float64
SegTSeas      float64
SegLowFlow    float64
DSDist        float64
DSMaxSlope    float64
USAvgT        float64
USRainDays    float64
USSlope       float64
USNative      float64
DSDam           int64
Method          int64
LocSed        float64
dtype: object

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [4]:
xgb_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    eval_metric="logloss",
    early_stopping_rounds=50,
    random_state=808
)

xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

best_trees = xgb_model.get_booster().best_iteration
print("Optimal number of trees:", best_trees)


[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[29]	validation_0-loglos

### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [5]:
# I couldn't get this to work right off the bat, so I tuned the learning rate without early stopping, and then added early stopping after


xgb_model_cv = XGBClassifier(
    n_estimators = 1000,
    learning_rate = 0.1,
    eval_metric = "logloss",
    random_state = 808
)

param_dist = {'learning_rate': uniform(0.01, 0.31)}

random_search = RandomizedSearchCV(
    estimator = xgb_model_cv,
    param_distributions = param_dist,
    n_iter = 20,
    cv = 5,
    random_state = 808,
    n_jobs = -1,
    verbose = 1
)

random_search.fit(X_train, y_train)
best_learning_rate = random_search.best_params_['learning_rate']


Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [6]:
final_model = XGBClassifier(
    n_estimators = 1000,
    learning_rate = best_learning_rate,
    eval_metric = "logloss",
    early_stopping_rounds = 50,
    random_state = 808
)

final_model.fit(
    X_train,
    y_train,
    eval_set = [(X_val, y_val)],
    verbose = True
)

best_learning_rate = final_model.get_params()['learning_rate']
print("Learning rate:", final_model.get_params()['learning_rate'])

[0]	validation_0-logloss:0.47593
[1]	validation_0-logloss:0.43478
[2]	validation_0-logloss:0.42251
[3]	validation_0-logloss:0.40614
[4]	validation_0-logloss:0.39372
[5]	validation_0-logloss:0.38916
[6]	validation_0-logloss:0.38863
[7]	validation_0-logloss:0.38720
[8]	validation_0-logloss:0.38200
[9]	validation_0-logloss:0.38325
[10]	validation_0-logloss:0.38283
[11]	validation_0-logloss:0.38274
[12]	validation_0-logloss:0.38611
[13]	validation_0-logloss:0.38398
[14]	validation_0-logloss:0.38586
[15]	validation_0-logloss:0.38827
[16]	validation_0-logloss:0.38801
[17]	validation_0-logloss:0.39105
[18]	validation_0-logloss:0.39026
[19]	validation_0-logloss:0.39329
[20]	validation_0-logloss:0.39838
[21]	validation_0-logloss:0.39845
[22]	validation_0-logloss:0.39714
[23]	validation_0-logloss:0.40215
[24]	validation_0-logloss:0.40440
[25]	validation_0-logloss:0.40766
[26]	validation_0-logloss:0.40765
[27]	validation_0-logloss:0.40949
[28]	validation_0-logloss:0.41147
[29]	validation_0-loglos

### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [7]:
tuned_model = XGBClassifier(
    n_estimators = best_trees,
    learning_rate = best_learning_rate,
    eval_metric = "logloss",
    random_state = 808
)

param_dist = {
    'max_depth': randint(3, 11),
    'min_child_weight': randint(1, 11),
    'gamma': uniform(0.05, 0.05)
}

random_search_complexity = RandomizedSearchCV(
    estimator = tuned_model,
    param_distributions = param_dist,
    n_iter = 20,
    cv = 5,
    random_state = 808,
    n_jobs = -1,
    verbose = 1
)

random_search_complexity.fit(X_train, y_train)

best_tree_parameters = random_search_complexity.best_params_

print("Best complexity parameters:", random_search_complexity.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best complexity parameters: {'gamma': 0.09077113838342292, 'max_depth': 6, 'min_child_weight': 9}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and 1.0 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and 1.0
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [8]:
final_model = XGBClassifier(
    n_estimators = best_trees,
    learning_rate = best_learning_rate,
    eval_metric = "logloss",
    random_state = 808,
    **best_tree_parameters
)

param_dist_stochastic = {
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5)
}

random_search_stochastic = RandomizedSearchCV(
    estimator = final_model,
    param_distributions = param_dist_stochastic,
    n_iter = 20,
    cv = 5,
    random_state = 808,
    n_jobs = -1,
    verbose = 1
)

random_search_stochastic.fit(X_train, y_train)

best_stochastic_params = random_search_stochastic.best_params_

print("Best stochastic parameters:", random_search_stochastic.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best stochastic parameters: {'colsample_bytree': 0.9365653342296307, 'subsample': 0.8785363858814997}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [9]:
eval_data['Method'] = le.transform(eval_data['Method'].astype(str))

X_eval = eval_data.drop("Angaus_obs", axis=1)
y_eval = eval_data["Angaus_obs"]

final_model = XGBClassifier(
    n_estimators=best_trees,
    learning_rate=best_learning_rate,
    eval_metric="logloss",
    random_state=808,
    **best_tree_parameters,
    **best_stochastic_params
)

final_model.fit(X_train, y_train)

predictions = final_model.predict(X_eval)

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [10]:
y_pred_proba = final_model.predict_proba(X_eval)[:, 1]

auc = roc_auc_score(y_eval, y_pred_proba)


importances = final_model.feature_importances_

for feature, importance in zip(X_eval.columns, importances):
    print(f"{feature}: {importance:.4f}")
    
print("AUC:", auc)

SegSumT: 0.2694
SegTSeas: 0.0528
SegLowFlow: 0.0454
DSDist: 0.0435
DSMaxSlope: 0.1112
USAvgT: 0.0822
USRainDays: 0.0836
USSlope: 0.0632
USNative: 0.0710
DSDam: 0.0000
Method: 0.1214
LocSed: 0.0565
AUC: 0.8636298780052792


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

For predictive performance, this model is very similar to Elith et al., with an AUC score of .863, where Elith had an AUC score of .858. However, Elith et al. also did an analysis on feature importance, which is key for an ecological understanding of, in this case, eels in New Zealand. The analysis on feature importance helps us interpret which features have the greatest influence. For this, Elith's model wins. 